In [3]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd 

headers = {'User Agent':'Mozilla/5.0'}

players = [
    'zion-williamson', 'temetrius-morant', 'rj-barrett', 'deandre-hunter', 'darius-garland', 'jarrett-culver',
    'coby-white', 'jaxon-hayes', 'rui-hachimura', 'cam-reddish', 'cameron-johnson-4', 'pj-washington', 
    'tyler-herro', 'romeo-langford', 'chuma-okeke', 'nickeil-alexander-walker', 'matisse-thybulle', 'brandon-clarke', 
    'grant-williams','ty-jerome', 'nassir-little', 'dylan-windler', 'mfiondu-kabengele', 'jordan-poole', 
    'keldon-johnson', 'kevin-porterjr', 'nic-claxton', 'kz-okpala', 'carsen-edwards', 'bruno-fernando', 
    'cody-martin', 'daniel-gafford', 'justin-james', 'eric-paschall', 'admiral-schofield', 'jaylen-nowell', 
    'bol-bol', 'isaiah-roby', 'talen-horton-tucker','ignas-brazdeikis', 'terance-mann', 'quinndary-weatherspoon', 
    'tremont-waters', 'jalen-mcdaniels', 'marial-shayok','kyle guy', 'jaylen-hands', 'jordan-bone', 
    'miye-oni', 'dewan-hernandez','caleb-martin', 'luguentz-dort', 'naz-reid', 'garrison-matthews']
   
player_stats = []
playerlist = []
year_list = []
age_list = []

for player in players:
    try:
        if player[-1].isdigit():
            url = f'https://www.sports-reference.com/cbb/players/{player}.html'
        else:
            url = f'https://www.sports-reference.com/cbb/players/{player}-1.html'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        header = [th.getText() for th in soup.findAll('tr', limit = 2)[0].findAll('th')]
        rows = soup.findAll('tr')
        player_stats.append([td.getText() for td in soup.find('tr', id ='players_per_game.2019')])
        table = pd.DataFrame(player_stats, columns = header) 
        for i in player_stats:
            if i[14] == '':
                i[14] = str(0.0)
        
        player = player.replace('-',' ').title()
        if player[-1].isdigit():
            player = player[:-2]
        if player[-2:] == 'jr':
            player = player.replace('jr', ' Jr')
        playerlist.append(player)
        
    except:
        continue
    try:
        url = f'https://www.google.com/search?q={player}+basketball+birth+date'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        age = (soup.findAll('div', class_ = 'BNeawe iBp4i AP7Wnd')[0])
        for x in age:
            x = x.text
        x = x.replace(',','').split()
        months = {
            'Janurary': '1',
            'February': '2',
            'March': '3',
            'April': '4',
            'May': '5',
            'June': '6',
            'July': '7',
            'August': '8',
            'September': '9',
            'October': '10',
            'November': '11',
            'December': '12'
        } 
        birth_date = months[x[0]]+ '-' + x[1] + '-' + x[2]
        draft_date = '7-1-2019'
        import datetime 
        birth_date_final = datetime.datetime.strptime(birth_date, '%m-%d-%Y')  
        draft_date_final = datetime.datetime.strptime(draft_date, '%m-%d-%Y')

        age_on_draft_night = draft_date_final - birth_date_final
        age = round(age_on_draft_night.days/365, 1)
        age_list.append(age)
        year = round(((1-(age/30))*2.95), 2)
        year_list.append(year)
        
    except:
        age_list.append(22)
        year_list.append(.85)


table.insert(0, "Name", playerlist)
table.insert(2, "Year", year_list)
table.insert(2, "Age", age_list)
table['MP'] = table['MP'].astype(float)
table['PTS'] = table['PTS'].astype(float)
table['AST'] = table['AST'].astype(float)
table['TRB'] = table['TRB'].astype(float)
table['BLK'] = table['BLK'].astype(float)
table['STL'] = table['STL'].astype(float)
table['3P'] = table['3P'].astype(float)
table['3PA'] = table['3PA'].astype(float)
table['TOV'] = table['TOV'].astype(float)
table['SOS'] = table['SOS'].astype(float)
table['PF'] = table['PF'].astype(float)
table['FT%'] = table['FT%'].astype(float)
table['3P%'] = table['3P%'].astype(float)

per40 = 40/table['MP']

table["Player Grade"] = ((table['PTS']*per40) + (table['TRB']*1.5*per40) + (table['AST']*2*per40) +
(table['BLK']*3*per40) + (table['STL']*3*per40) + (table['3P']*5*per40)+ (table['FT%']*7) + (table['SOS']) + 
(table['3P%']*10)) * table['Year']

table["Player Grade"] = table["Player Grade"]*1.25
table["Player Grade"] = (round(table["Player Grade"], 1))
table["Player Grade"]= table["Player Grade"].astype(float)
                        
table = table.sort_values(by= "Player Grade", ascending=False)
table.insert(5, 'SoS', table['SOS'])
table.reset_index(drop = True, inplace=True)
import numpy as np
table.index = np.arange(1, len(table)+1)
del table['SOS']
del table['Conf']
del table['ORB']
del table['DRB']
del table['GS']
del table['FG']
del table['FGA']
del table['FT']
del table['FTA']
del table['2P']
del table['2PA']
del table['2P%']
del table['Year']
del table['Age']
table

,Name,Season,School,SoS,G,MP,FG%,3P,3PA,3P%,FT%,TRB,AST,STL,BLK,TOV,PF,PTS,,Player Grade
1,Zion Williamson,2018-19,Duke,11.98,33,30.0,.680,0.7,2.2,0.338,0.640,8.9,2.1,2.1,1.8,2.4,2.1,22.6,,126.4
2,Bol Bol,2018-19,Oregon,6.13,9,29.8,.561,1.4,2.8,0.520,0.757,9.6,1.0,0.8,2.7,2.0,1.7,21.0,,115.2
3,Rj Barrett,2018-19,Duke,11.98,38,35.3,.454,1.9,6.2,0.308,0.665,7.6,4.3,0.9,0.4,3.2,1.8,22.6,,111.2
4,Coby White,2018-19,UNC,11.35,35,28.5,.423,2.3,6.6,0.353,0.800,3.5,4.1,1.1,0.3,2.7,1.9,16.1,,109.2
5,Temetrius Morant,2018-19,Murray State,-3.11,33,36.6,.499,1.7,4.8,0.363,0.813,5.7,10.0,1.8,0.8,5.2,1.5,24.5,,101.5
6,Cam Reddish,2018-19,Duke,11.98,36,29.7,.356,2.5,7.4,0.333,0.772,3.7,1.9,1.6,0.6,2.7,2.4,13.5,,96.5
7,Carsen Edwards,2018-19,Purdue,11.99,36,35.4,.394,3.8,10.6,0.355,0.837,3.6,2.9,1.3,0.3,3.1,2.0,24.3,,95.0
8,Nassir Little,2018-19,UNC,11.35,36,18.2,.478,0.4,1.4,0.269,0.770,4.6,0.7,0.5,0.5,1.3,1.6,9.8,,91.3
9,Jarrett Culver,2018-19,Texas Tech,9.53,38,32.5,.461,1.3,4.2,0.304,0.707,6.4,3.7,1.5,0.6,2.7,1.9,18.5,,90.4
10,Jaylen Hands,2018-19,UCLA,6.78,33,31.2,.413,2.0,5.4,0.373,0.780,3.7,6.1,1.3,0.2,3.2,1.2,14.2,,88.7
